In [4]:
import numpy as np

A = np.array([[0,0,2,0,0,0,0,0],
              [0,0,3,0,0,0,0,0],
              [0,0,0,4,0,0,0,0],
              [0,0,0,0,5,1,8,0],
              [0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,2,0],
              [0,0,0,0,0,0,0,5],
              [0,0,0,0,0,0,0,0]
             ])

B = np.array([
        [0,0,0,0,4,0,0,7,2],
        [0,0,0,0,0,0,0,0,0],
        [0,1,0,0,0,0,3,0,0],
        [0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,2,0,0],
        [2,0,0,0,0,0,0,0,0],
        [0,0,0,5,0,0,0,0,0],
        [0,0,0,0,0,0,0,3,0]
    ])

A_in_w = np.sum(A, axis=0)
A_out_w = np.sum(A, axis=1)
B_in_w = np.sum(B, axis=0)
B_out_w = np.sum(B, axis=1)

A_in = np.sum(A>0, axis=0)
A_out = np.sum(A>0, axis=1)
B_in = np.sum(B>0, axis=0)
B_out = np.sum(B>0, axis=1)

# 0 => peripheral, 1 => bridge, 2 => center
A_role = np.array([0,0,1,2,0,0,0,0])
B_role = np.array([2,0,0,0,0,0,1,0,0])


def ego_connectivity(G):
    ego_conn = np.zeros((G.shape[0]))
    for i in range(G.shape[0]):
        ego_i = (G[i] | G.T[i])>0
        ego_i[i] = True
        ego_net = G[ego_i].T[ego_i]
        ego_conn[i] = np.sum(ego_net > 0)/float(ego_net.shape[0]**2)
    return ego_conn


A_ego_conn = ego_connectivity(A);
B_ego_conn = ego_connectivity(B);
print(A_ego_conn)
print(B_ego_conn)

A_feature = np.array([A_in, A_out, A_in_w, A_out_w, A_ego_conn])
print(A.shape)

[ 0.25        0.25        0.1875      0.2         0.25        0.33333333
  0.25        0.25      ]
[ 0.2         0.25        0.22222222  0.25        0.25        0.25        0.1875
  0.25        0.33333333]
(8, 8)


In [1]:
# produce graph A
import numpy as np
nodes = []
with open('facebook/0.edges') as f:
    for line in f:
        pair = line.strip().split()
        if int(pair[0]) == int(pair[1]):
            print("hey\n")
        nodes.append(int(pair[0]))
        nodes.append(int(pair[1]))
    
A_size = max(nodes)+1
A = np.zeros((A_size,A_size))

for i in range(0,len(nodes)-2,2):
    A[nodes[i]][nodes[i+1]]=1
    A[nodes[i+1]][nodes[i]]=1
    
A[0,:]=1
A[:,0]=1
A[0,0]=0

A_feats = np.zeros((A_size, 224))

with open('facebook/0.feat') as f:
    for line in f:
        feats = line.strip().split()
        A_feats[int(feats[0])] = feats[1:]
        
print(A_feats[3])

[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.
  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.]


In [2]:
# produce permuted B with noise
perm = np.random.permutation(len(A))

P = np.identity(len(A))
P = P[perm]

B = P.dot(A).dot(P.T)
#noise = np.random.choice([0, 1], size=(len(A),len(A)), p=[99.95/100, 0.05/100])
#B = (B + noise + noise.T)%2

print("noise edges: " + str(np.sum(np.abs(A-P.T.dot(B).dot(P))[:])))

with open('B.edges', 'w') as f:
    for i in range(len(B)):
        for j in range(i+1, len(B)):
            if B[i][j]>0:
                f.write(str(i)+" "+str(j)+"\n")

noise edges: 0.0


In [73]:
def loadGraph(fname):
    nodes = []
    with open(fname) as f:
        for line in f:
            pair = line.strip().split()
            if int(pair[0]) == int(pair[1]):
                print("hey\n")
            nodes.append(int(pair[0]))
            nodes.append(int(pair[1]))

    A_size = max(nodes)+1
    A = np.zeros((A_size,A_size))

    for i in range(0,len(nodes),2):
        A[nodes[i]][nodes[i+1]]=1
        A[nodes[i+1]][nodes[i]]=1
    
    #A[0,:]=1
    #A[:,0]=1
    #A[0,0]=0
    return A

A = loadGraph('0.edges')

def permuteNoiseMat(A, is_perm=False, has_noise=False):
    perm = np.random.permutation(len(A))

    P = np.identity(len(A))
    if is_perm:
        P = P[perm]

    B = P.dot(A).dot(P.T)
    if has_noise:
        noise = np.random.choice([0, 1], size=(len(A),len(A)), p=[99.95/100, 0.05/100])
        B = (B + noise + noise.T)%2
    
    return B

B = permuteNoiseMat(A,False,True)

with open('B.edges', 'w') as f:
    for i in range(len(B)):
        for j in range(i+1, len(B)):
            if B[i][j]>0:
                f.write(str(i)+" "+str(j)+"\n")

In [74]:
import snap
import numpy as np
import pandas as pd

def getAttribute(filename):
    UGraph = snap.LoadEdgeList(snap.PUNGraph, filename, 0, 1)
    UGraph.Dump()

    attributes = pd.DataFrame(np.zeros(shape=(UGraph.GetNodes(), 12)), 
                              columns=['Graph', 'Id', 'Degree', 'DegreeCentrality', 'NodeBetweennessCentrality', 
                                       'ClosenessCentrality', 'FarnessCentrality', 'PageRank', 'HubsScore', 
                                       'AuthoritiesScore', 'NodeEccentricity', 'EigenvectorCentrality'])
    
    attributes['Graph'] = [filename] * UGraph.GetNodes()
    
    # Degree
    id = []
    degree = []
    OutDegV = snap.TIntPrV()
    snap.GetNodeOutDegV(UGraph, OutDegV)
    for item in OutDegV:
        id.append(item.GetVal1())
        degree.append(item.GetVal2())
    attributes['Id'] = id
    attributes['Degree'] = degree

    # Degree, Closeness, Farness Centrality, Node Eccentricity
    degCentr = []
    cloCentr = []
    farCentr = []
    nodeEcc = []
    for NI in UGraph.Nodes():
        degCentr.append(snap.GetDegreeCentr(UGraph, NI.GetId()))
        cloCentr.append(snap.GetClosenessCentr(UGraph, NI.GetId()))
        farCentr.append(snap.GetFarnessCentr(UGraph, NI.GetId()))
        nodeEcc.append(snap.GetNodeEcc(UGraph, NI.GetId(), False))
    attributes['DegreeCentrality'] = degCentr
    attributes['ClosenessCentrality'] = cloCentr
    attributes['FarnessCentrality'] = farCentr
    attributes['NodeEccentricity'] = nodeEcc

    # Betweenness Centrality
    betCentr = []
    Nodes = snap.TIntFltH()
    Edges = snap.TIntPrFltH()
    snap.GetBetweennessCentr(UGraph, Nodes, Edges, 1.0)
    for node in Nodes:
        betCentr.append(Nodes[node])
    attributes['NodeBetweennessCentrality'] = betCentr

    # PageRank
    pgRank = []
    PRankH = snap.TIntFltH()
    snap.GetPageRank(UGraph, PRankH)
    for item in PRankH:
        pgRank.append(PRankH[item])
    attributes['PageRank'] = pgRank

    # Hubs, Authorities score 
    hubs = []
    auth = []
    NIdHubH = snap.TIntFltH()
    NIdAuthH = snap.TIntFltH()
    snap.GetHits(UGraph, NIdHubH, NIdAuthH)
    for item in NIdHubH:
        hubs.append(NIdHubH[item])
    for item in NIdAuthH:
        auth.append(NIdAuthH[item])
    attributes['HubsScore'] = hubs
    attributes['AuthoritiesScore'] = auth

    # Eigenvector Centrality
    eigenCentr = []
    NIdEigenH = snap.TIntFltH()
    snap.GetEigenVectorCentr(UGraph, NIdEigenH)
    for item in NIdEigenH:
        eigenCentr.append(NIdEigenH[item])
    attributes['EigenvectorCentrality'] = eigenCentr

    return attributes

attributesA = getAttribute("0.edges")
with open('attributesA', 'w') as f:
    for index, row in attributesA.iterrows():
        f.write(str(attributesA.ix[index]))
print attributesA

attributesB = getAttribute("B.edges")
with open('attributesB', 'w') as f:
    for index, row in attributesB.iterrows():
        f.write(str(attributesB.ix[index]))
print attributesB

       Graph   Id  Degree  DegreeCentrality  NodeBetweennessCentrality  \
0    0.edges  236      36          0.108434                 200.150607   
1    0.edges  186      43          0.129518                 302.943946   
2    0.edges  122      62          0.186747                1250.238251   
3    0.edges  285      46          0.138554                 752.212911   
4    0.edges   24      15          0.045181                   5.902781   
5    0.edges  346      26          0.078313                 255.675531   
6    0.edges  271      72          0.216867                 822.781836   
7    0.edges  304      54          0.162651                 744.431882   
8    0.edges  176      13          0.039157                   8.938337   
9    0.edges    9      56          0.168675                 541.363249   
10   0.edges  130      15          0.045181                 548.399270   
11   0.edges  329      29          0.087349                1754.716168   
12   0.edges  204      21          0.0

In [75]:
import numpy as np
from collections import defaultdict

def generateBuckets(attributes, cols):
    randMatrix = np.random.random((attributes.shape[1], cols))
    randMatrix = np.multiply(((randMatrix - 0.5) * 2).T, 1 / np.median(attributes, axis=0)).T
    #randMatrix = (randMatrix - 0.5) * 2
    signMatrix = attributes.dot(randMatrix)
    signMatrix[signMatrix > 0] = 1
    signMatrix[signMatrix < 0] = 0
    hashMatrix = signMatrix.dot([2**i for i in range(cols)])
    dic = defaultdict(list)
    for i in range(len(hashMatrix)):
        dic[hashMatrix[i]].append(i)
    return dic

combineAB = np.vstack((attributesA.as_matrix(), attributesB.as_matrix()))

band1 = ['Degree','NodeBetweennessCentrality','FarnessCentrality','NodeEccentricity']
bucket = generateBuckets(combineAB[:, [2,4,6,10]], 20)
with open('buckets-band-1', 'w') as f:
    for k, v in bucket.items():
        f.write(str(k) + str(v) + '\n')
print len(bucket.items())

band2 = ['DegreeCentrality','ClosenessCentrality','PageRank', 'HubsScore','AuthoritiesScore','EigenvectorCentrality']
band2 = np.vstack((attributesA[band2].as_matrix(), attributesB[band2].as_matrix()))
print len(generateBuckets(band2, 20).items())

50
48


In [25]:
a = [[1,2,3,4],[2,3,4,5]]
print a
print a > 3

[[1, 2, 3, 4], [2, 3, 4, 5]]
True


In [78]:
print combineAB[129]
print combineAB[450]
print combineAB[477]
print combineAB[513]

['0.edges' 13 30 0.09036144578313253 79.64637142493234 0.33644965300172847
 2.9722129034113243 0.00389268630632554 0.09668888704012608
 0.0966888870401251 7 0.09668848152226041]
['B.edges' 232 24 0.07142857142857142 66.06372757704608 0.36270790296096195
 2.7570394574711803 0.0033396218322273644 0.07577413046847993
 0.07577413046847986 6 0.07577409117408997]
['B.edges' 211 29 0.08630952380952381 94.71271071986091 0.36392232674319724
 2.747839103330565 0.003937816526265188 0.08015530732416427
 0.08015530732416337 6 0.08015492117603855]
['B.edges' 104 31 0.09226190476190477 88.0506967978818 0.36392232674319724
 2.747839103330565 0.004146204063325271 0.10157095408558175
 0.10157095408558142 6 0.10157077780789853]
